# LlamaIndex: Structured Data Extraction

- Structured Data Extraction<br>
  - [Introduction to Structured Data Extraction]()
  - []()
  - []()
  - []()

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## [Introduction to Structured Data Extraction
The core of the way structured data extraction works in LlamaIndex is **Pydantic** classes:<br>
you define a data structure in Pydantic and LlamaIndex works with Pydantic to coerce the output of the LLM into that structure.
- https://docs.pydantic.dev/latest/

### What is Pydantic?
Pydantic is a widely-used data validation and conversion library.<br>
It relies heavily on Python type declarations.<br>
There is an extensive **guide to Pydantic** in that project’s documentation
- https://docs.pydantic.dev/latest/concepts/models/

In [2]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str = "Jane Doe"

In [3]:
from typing import List, Optional
from pydantic import BaseModel

class Foo(BaseModel):
    count: int
    size: Optional[float] = None

class Bar(BaseModel):
    apple: str = "x"
    banana: str = "y"

class Spam(BaseModel):
    foo: Foo
    bars: List[Bar]

### Converting Pydantic objects to JSON schemas
Pydantic supports converting Pydantic classes into **JSON-serialized schema objects** which conform to popular standards.
- https://docs.pydantic.dev/latest/concepts/json_schema/

```json
{
  "properties": {
    "id": {
      "title": "Id",
      "type": "integer"
    },
    "name": {
      "default": "Jane Doe",
      "title": "Name",
      "type": "string"
    }
  },
  "required": [
    "id"
  ],
  "title": "User",
  "type": "object"
}
```

In [4]:
import json
from enum import Enum
from typing import Annotated

from pydantic import BaseModel, Field
from pydantic.config import ConfigDict

class FooBar(BaseModel):
    count: int
    size: float | None = None

class Gender(str, Enum):
    male = 'male'
    female = 'female'
    other = 'other'
    not_given = 'not_given'

class MainModel(BaseModel):
    """
    This is the description of the main model
    """

    model_config = ConfigDict(title='Main')

    foo_bar: FooBar
    gender: Annotated[Gender | None, Field(alias='Gender')] = None
    snap: int = Field(
        default=42,
        title='The Snap',
        description='this is the value of snap',
        gt=30,
        lt=50,
    )

main_model_schema = MainModel.model_json_schema()  # (1)!
print(json.dumps(main_model_schema, indent=2))  # (2)!

{
  "$defs": {
    "FooBar": {
      "properties": {
        "count": {
          "title": "Count",
          "type": "integer"
        },
        "size": {
          "anyOf": [
            {
              "type": "number"
            },
            {
              "type": "null"
            }
          ],
          "default": null,
          "title": "Size"
        }
      },
      "required": [
        "count"
      ],
      "title": "FooBar",
      "type": "object"
    },
    "Gender": {
      "enum": [
        "male",
        "female",
        "other",
        "not_given"
      ],
      "title": "Gender",
      "type": "string"
    }
  },
  "description": "This is the description of the main model",
  "properties": {
    "foo_bar": {
      "$ref": "#/$defs/FooBar"
    },
    "Gender": {
      "anyOf": [
        {
          "$ref": "#/$defs/Gender"
        },
        {
          "type": "null"
        }
      ],
      "default": null
    },
    "snap": {
      "default": 42,
      "

### Using annotations
As mentioned, LLMs are using JSON schemas from Pydantic as instructions on how to return data.<br>
To assist them and improve the accuracy of your returned data, it’s helpful to include natural-language descriptions of objects and fields and what they’re used for.<br>
Pydantic has support for this with **docstrings** and **Fields**.
- https://www.geeksforgeeks.org/python-docstrings/
- https://docs.pydantic.dev/latest/concepts/fields/

In [5]:
from datetime import datetime

class LineItem(BaseModel):
    """A line item in an invoice."""

    item_name: str = Field(description="The name of this item")
    price: float = Field(description="The price of this item")

class Invoice(BaseModel):
    """A representation of information from an invoice."""

    invoice_id: str = Field(
        description="A unique identifier for this invoice, often a number"
    )
    date: datetime = Field(description="The date this invoice was created")
    line_items: list[LineItem] = Field(
        description="A list of all the items in this invoice"
    )

```json
{
  "$defs": {
    "LineItem": {
      "description": "A line item in an invoice.",
      "properties": {
        "item_name": {
          "description": "The name of this item",
          "title": "Item Name",
          "type": "string"
        },
        "price": {
          "description": "The price of this item",
          "title": "Price",
          "type": "number"
        }
      },
      "required": [
        "item_name",
        "price"
      ],
      "title": "LineItem",
      "type": "object"
    }
  },
  "description": "A representation of information from an invoice.",
  "properties": {
    "invoice_id": {
      "description": "A unique identifier for this invoice, often a number",
      "title": "Invoice Id",
      "type": "string"
    },
    "date": {
      "description": "The date this invoice was created",
      "format": "date-time",
      "title": "Date",
      "type": "string"
    },
    "line_items": {
      "description": "A list of all the items in this invoice",
      "items": {
        "$ref": "#/$defs/LineItem"
      },
      "title": "Line Items",
      "type": "array"
    }
  },
  "required": [
    "invoice_id",
    "date",
    "line_items"
  ],
  "title": "Invoice",
  "type": "object"
}
```

In [6]:
main_model_schema = Invoice.model_json_schema()  # (1)!
print(json.dumps(main_model_schema, indent=2))  # (2)!

{
  "$defs": {
    "LineItem": {
      "description": "A line item in an invoice.",
      "properties": {
        "item_name": {
          "description": "The name of this item",
          "title": "Item Name",
          "type": "string"
        },
        "price": {
          "description": "The price of this item",
          "title": "Price",
          "type": "number"
        }
      },
      "required": [
        "item_name",
        "price"
      ],
      "title": "LineItem",
      "type": "object"
    }
  },
  "description": "A representation of information from an invoice.",
  "properties": {
    "invoice_id": {
      "description": "A unique identifier for this invoice, often a number",
      "title": "Invoice Id",
      "type": "string"
    },
    "date": {
      "description": "The date this invoice was created",
      "format": "date-time",
      "title": "Date",
      "type": "string"
    },
    "line_items": {
      "description": "A list of all the items in this invoice",

## Using Structured LLMs

In [7]:
from datetime import datetime

class LineItem(BaseModel):
    """A line item in an invoice."""

    item_name: str = Field(description="The name of this item")
    price: float = Field(description="The price of this item")

class Invoice(BaseModel):
    """A representation of information from an invoice."""

    invoice_id: str = Field(
        description="A unique identifier for this invoice, often a number"
    )
    date: datetime = Field(description="The date this invoice was created")
    line_items: list[LineItem] = Field(
        description="A list of all the items in this invoice"
    )

Note: for better parsing of PDFs, we recommend **LlamaParse**
- https://docs.cloud.llamaindex.ai/llamaparse/getting_started

In [8]:
from llama_index.readers.file import PDFReader
from pathlib import Path

pdf_reader = PDFReader()
# documents = pdf_reader.load_data(file=Path("./uber_receipt.pdf"))
documents = pdf_reader.load_data(file=Path("./data/uber_2021.pdf"))  # 307 pages
print(len(documents))

307


In [9]:
text = documents[0].text

In [10]:
from llama_index.llms.openai import OpenAI

# llm = OpenAI(model="gpt-4o")
llm = OpenAI(model="gpt-4o-mini")
sllm = llm.as_structured_llm(Invoice)

response = sllm.complete(text)

In [11]:
json_response = json.loads(response.text)
print(json.dumps(json_response, indent=2))

{
  "invoice_id": "001-38902",
  "date": "2021-12-31T00:00:00Z",
  "line_items": [
    {
      "item_name": "Common Stock",
      "price": 1e-05
    }
  ]
}


In [12]:
print(text)

UNITED STATES
SECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549 ____________________________________________ FORM 10-K____________________________________________ (Mark One)☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the fiscal year ended  December 31, 2021OR☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934For the transition period from_____ to _____            Commission File Number: 001-38902____________________________________________ UBER TECHNOLOGIES, INC.(Exact name of registrant as specified in its charter)____________________________________________ Delaware45-2647441 (State or other jurisdiction of incorporation or organization)(I.R.S. Employer Identification No.) 1515 3rd StreetSan Francisco, California 94158(Address of principal executive offices, including zip code)(415) 612-8582 (Registrant’s telephone number, including area code) ____________________________________________Securi

In [13]:
from pprint import pprint

pprint(response.raw)

Invoice(invoice_id='001-38902', date=datetime.datetime(2021, 12, 31, 0, 0, tzinfo=TzInfo(UTC)), line_items=[LineItem(item_name='Common Stock', price=1e-05)])


In [14]:
print(response.raw)

invoice_id='001-38902' date=datetime.datetime(2021, 12, 31, 0, 0, tzinfo=TzInfo(UTC)) line_items=[LineItem(item_name='Common Stock', price=1e-05)]
